In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import numpy as np
import pandas as pd
from scipy.interpolate import griddata, interp1d
from torch.utils.data import DataLoader
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import sklearn.model_selection as ms
from IPython.core.display import display, HTML
np.set_printoptions(precision=6, suppress=True)
from pprz_data.pprz_data import DATA
import torch
import torch.nn as nn
from torch.autograd import Variable

In [3]:
def assign_fault(df):
    df = df.assign(fault=0)
    cond1 = (df['add1'] > 0.005) | (df['add1'] < -0.005)
    cond2 = (df['add2'] > 0.005) | (df['add2'] < -0.005)
    cond3 = (df['m1'] < 1.0) | (df['m2'] < 1.0)
    cond = cond1 | cond2 | cond3
    df.loc[cond,'fault'] = 1
    return df




ac_id='20'

filename = "/home/jovyan/code/fault_detection-master/data/21_07_2020_Faulty_Daredevil_MultiClass_Sweep/20_07_21__07_00_27_SD.data"
sd21 = DATA(filename, ac_id, data_type="flight", sample_period=0.1)
# GCS data
df_21 = sd21.get_labelled_data()

filename = "/home/jovyan/code/fault_detection-master/data/23_07_2020_Faulty_Daredevil_MultiClass_Sweep_2/20_07_23__07_19_26_SD.data"
sd23 = DATA(filename, ac_id, data_type="flight", sample_period=0.1)
# Labelled data can be built from the data class directly
df_23 = sd23.get_labelled_data()

               phi       psi     theta
time                                  
34.662   -0.012592  3.134684 -0.075622
34.712   -0.012572  3.134683 -0.075622
34.762   -0.012580  3.134644 -0.075633
34.812   -0.012564  3.134672 -0.075642
34.862   -0.012586  3.134583 -0.075640
...            ...       ...       ...
3034.862 -0.018029  2.218574 -0.074966
3034.912 -0.018051  2.218460 -0.074943
3034.962 -0.018019  2.218359 -0.074911
3035.012 -0.018081  2.218320 -0.074884
3035.062 -0.018073  2.218254 -0.074857

[60009 rows x 3 columns]
          mode
time          
34.912     0.0
35.912     0.0
36.912     0.0
37.912     0.0
38.912     0.0
...        ...
3030.912   0.0
3031.912   0.0
3032.912   1.0
3033.912   0.0
3034.912   0.0

[3001 rows x 1 columns]
                Ax        Ay        Az
time                                  
34.632    0.365234  0.159180 -9.747070
34.652    0.378906  0.137695 -9.727539
34.672    0.396484  0.114258 -9.756836
34.692    0.383789  0.068359 -9.758789
34.712    0.3

          mode
time          
34.932     0.0
35.932     0.0
36.932     0.0
37.932     0.0
38.932     0.0
...        ...
3275.932   0.0
3276.932   0.0
3277.932   0.0
3278.932   0.0
3279.932   0.0

[3246 rows x 1 columns]
                Ax        Ay        Az
time                                  
34.652    0.367188  0.159180 -9.751953
34.672    0.396484  0.207031 -9.725586
34.692    0.381836  0.195312 -9.742188
34.712    0.405273  0.162109 -9.744141
34.732    0.372070  0.159180 -9.723633
...            ...       ...       ...
3280.692  0.469727  0.112305 -9.614258
3280.712  0.469727  0.097656 -9.625977
3280.732  0.469727  0.119141 -9.659180
3280.752  0.479492  0.107422 -9.645508
3280.772  0.462891  0.102539 -9.675781

[162307 rows x 3 columns]
                east        north  course       alt  vel  climb
time                                                           
34.732    36027837.0  481360677.0     0.0  188829.0  0.0    0.0
34.932    36027865.0  481360683.0     0.0  188566.0  0

In [4]:

df_21 = assign_fault(df_21)

cond = (df_21['m2'] < 1.0)   # df.loc[cond,'fault'] = 1
df_21.fault[cond] = 2
cond = (df_21['m2'] < 0.9); df_21.fault[cond] = 3
cond = (df_21['m2'] < 0.8); df_21.fault[cond] = 4
cond = (df_21['m2'] < 0.7); df_21.fault[cond] = 5
cond = (df_21['m2'] < 0.6); df_21.fault[cond] = 6
cond = (df_21['m2'] < 0.5); df_21.fault[cond] = 7
cond = (df_21['m2'] < 0.4); df_21.fault[cond] = 8
flight = df_21[400:2600].copy() 

df_23 = assign_fault(df_23)

cond = (df_23['m2'] < 1.0)   # df.loc[cond,'fault'] = 1
df_23.fault[cond] = 2
cond = (df_23['m2'] < 0.9); df_23.fault[cond] = 3
cond = (df_23['m2'] < 0.8); df_23.fault[cond] = 4
cond = (df_23['m2'] < 0.7); df_23.fault[cond] = 5
cond = (df_23['m2'] < 0.6); df_23.fault[cond] = 6
cond = (df_23['m2'] < 0.5); df_23.fault[cond] = 7
cond = (df_23['m2'] < 0.4); df_23.fault[cond] = 8

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: Setti

In [6]:

def add_time_history_1(X,y,n_step=3,t_slide=1):
    '''X[0] = Ax0Ay0Az0Gx0Gy0Gz0Ax1Ay1Az1Gx1Gy1Gz1...'''
    
    time_len= X.shape[0] #-n_step        
    column_len = X.shape[1]        
    P=len(np.arange(n_step,time_len,t_slide))    
    xx=np.zeros((P+n_step,column_len*n_step))
    
    for i in range(n_step,P+n_step):
        for j in range(n_step):
            xx[i,j*column_len:(j+1)*column_len] = X[t_slide*i-t_slide*n_step+j]  

    xx = xx[n_step:X.shape[0],:]        
       
    yy = y[[np.arange(n_step,time_len,t_slide)-1]]  
    print(xx.shape)
    return xx,yy


from sklearn.preprocessing import MinMaxScaler
class Time():
    def __init__(self, start, end):
        self.start = start
        self.end = end

In [7]:
# A1
seq_len, input_dim = 128, 20
num_heads, hidden_dim = 8, 128
dropout = 0.1
class MultiHeadAttention2(nn.Module):
    def __init__(self, input_dim, num_heads, hidden_dim, hidden ,dropout):
        super(MultiHeadAttention2, self).__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, input_dim)
        self.fc1 = nn.Linear(hidden, input_dim)
        self.dropout_layer = nn.Dropout(dropout)
      
    def forward(self, x, mask=None):
        batch_size, seq_len, input_dim = x.size()        

        query = self.query(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        key = self.key(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)
        value = self.value(x).view(batch_size, seq_len, self.num_heads, -1).transpose(1, 2)       

        dot_product = torch.matmul(query, key.transpose(-1, -2)) / ((self.hidden_dim/num_heads)** 0.5)
        attention_scores = torch.softmax(dot_product, dim=-1)      
        attention_scores = self.dropout_layer(attention_scores)

        weighted_sum = torch.matmul(attention_scores, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
        return weighted_sum
    
    
import torch.nn.utils as utils 

class ASU(nn.Module):
    def __init__(self,size,batch_size,size2, hidden):
        super(ASU, self).__init__()        
        self.matrix1 = utils.weight_norm(nn.Linear(size,size), name='weight')
        self.matrix2 = utils.weight_norm(nn.Linear(size,size), name='weight')
#         self.matrix1 = nn.Linear(size,size)
#         self.matrix2 = nn.Linear(size,size)
        
        self.Attention = MultiHeadAttention2(size2,num_heads,size2, hidden, dropout)
        
    def forward(self, input1, input2):

        input_reshaped = torch.cat((input1.transpose(1, 2), input2), dim=-1)
        output_reshaped0 = self.matrix1(input_reshaped)
        output_reshaped0 = self.matrix2(output_reshaped0)        
        output_reshaped=self.Attention(output_reshaped0).view(output_reshaped0.shape)        
        output1 = output_reshaped[:, :,:input2.shape[2]].reshape(input1.shape)        
        output2 = output_reshaped[:,: ,input2.shape[2]:].view(input2.shape)
        
        
        return output1, output2,_

In [8]:
INPUT_SIZE=12
class HDMTLNet(nn.Module):
    def __init__(self,output_dim,ASU_enabled):
        super(HDMTLNet, self).__init__()
        self.conv1_1 = nn.Sequential(
            nn.Conv1d(20, 128, 3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool1d(20),
        )
        
        self.conv2_2 = nn.Sequential(
            nn.Conv1d(128, 256, 3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
        )                 
        self.conv3_3 = nn.Sequential(
            nn.Conv1d(256, 512, 3, padding=1),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool1d(1),
        )
        

        
        self.rnn1_1 = nn.LSTM(         
            input_size=INPUT_SIZE,
            hidden_size=128,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       

        )
        
        self.rnn2_2 = nn.LSTM(         
            input_size=128,
            hidden_size=256,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,      

        )
        self.rnn3_3 = nn.LSTM(         
            input_size=256,
            hidden_size=512,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,      
        )
                            
       
        self.Batch1_1 = nn.Sequential(

            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )
        self.Batch2_2 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )
        
        self.Batch3_3 = nn.Sequential(
            nn.BatchNorm1d(20),
            nn.ReLU(inplace=True),
        )  
        
        self.layer1 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Linear(256, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, output_dim),
        )
        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.Linear(256, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, output_dim),
        )
        

        self.ASU_enabled = ASU_enabled
        
        self.ASU1 = ASU(256,batch_size,256,384)
        self.ASU2 = ASU(512,batch_size,512,640)
        self.ASU3 = ASU(1024,batch_size,1024,1152)

    def forward(self, x,epoch):


        cross_start=0
        conv1_1 = self.conv1_1(x)
        rnn1_1,(h_n, h_c) = self.rnn1_1(x, None)
        rnn1_1=self.Batch1_1(rnn1_1)
        if self.ASU_enabled:
            if epoch > cross_start:
                crossC1_1, crossR1_1,matrix = self.ASU1(conv1_1, rnn1_1)
            else:
                crossC1_1, crossR1_1 = conv1_1, rnn1_1
        else:
            crossC1_1, crossR1_1 = conv1_1, rnn1_1
            

        conv2_2 = self.conv2_2(crossC1_1)
        rnn2_2,(h_n, h_c) = self.rnn2_2(crossR1_1, None)   
        rnn2_2=self.Batch2_2(rnn2_2)    

        if self.ASU_enabled:
            if epoch > cross_start:
                crossC2_2, crossR2_2,matrix = self.ASU2(conv2_2, rnn2_2)
            else:
                crossC2_2, crossR2_2 = conv2_2, rnn2_2
        else:
            crossC2_2, crossR2_2 = conv2_2, rnn2_2
            

        conv3_3 = self.conv3_3(crossC2_2)
        rnn3_3,(h_n, h_c) = self.rnn3_3(crossR2_2, None)
        rnn3_3=self.Batch3_3(rnn3_3)    
        rnn3_3 = rnn3_3[:, -1, :].reshape(batch_size,1,-1)

        if self.ASU_enabled:
            if epoch > cross_start:
                
                crossC3_3, crossR3_3,matrix = self.ASU3(conv3_3, rnn3_3)
            else:
                
                crossC3_3, crossR3_3 = conv3_3, rnn3_3               
        else:
            crossC3_3, crossR3_3 = conv3_3, rnn3_3
             
        fc2_1 = self.layer1(crossC3_3.squeeze())                
        fc2_2 = self.layer2(crossR3_3.squeeze())

        return fc2_1, fc2_2

In [9]:

def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            print("can't create directory '{}''".format(path))
            
            exit(1)
            
def SampleProduction(df, time=None, features=['airspeed', 'phi','psi','theta','Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz', 'C1', 'C2'], targets=['nominal', 'faulty']):
    # Create the Feature and Label List
    columns=features
    if time==None:
        flight = df.copy()
    else:
        flight = df[time.start:time.end].copy()                                                                                                                     
    X_pre = flight[columns].values # Features          
    y = flight.fault.values   # Labels
    X_sd,y_sd = add_time_history_1(X_pre,y,n_step= 20)
    
    X_train, X_test, y_train, y_test = ms.train_test_split(X_sd, y_sd, test_size=0.1, random_state=42, stratify=y_sd)

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train.reshape(-1,20,12), X_test.reshape(-1,20,12), y_train, y_test

def Cross_train_and_test(exp_name,exps):
    
    X_train0, X_test0, y_train0, y_test0 = SampleProduction(df_23, time, targets = ['nominal', 'R0.3', 'L0.9', 'L0.8','L0.7', 'L0.6', 'L0.5', 'L0.4', 
                                                                               'L0.3']  )
    X_train0=torch.as_tensor(X_train0, dtype=None, device=None)
    
    X_train0 = X_train0.float().cuda()
    
    X_test0=torch.as_tensor(X_test0, dtype=None, device=None)
    X_test0 = X_test0.float().cuda()
    
    train_classes = sorted(list(set(y_train0)))
    
    train_indices = [np.where(y_train0 == i)[0] for i in train_classes]    
    
    for exp in exps:
        scores1=[]
        scores2=[]
        num = int(exp/len(train_classes))        
        for time_idx in range (times):
            np.random.seed(0)
            a=HDMTLNet(9,ASU_enabled).to(device)
            criterion = nn.CrossEntropyLoss()

            optimizer = optim.Adam([
                {'params': (p for name, p in a.named_parameters() if 'bias'not in name), 'weight_decay': 0.001},
                {'params': (p for name, p in a.named_parameters() if 'bias' in name)},
                ],  lr=0.00001)
    
            settings["save_path"] = "Result_2/%s/size_%s/time_%s/" % (exp_name,exp,time_idx)
            
            mkdir(settings["save_path"])
            
            train_idxs = []
            val_idxs = []
            print("*************%s/size_%s/time_%s/" % (exp_name,exp,time_idx))
            
            for i, c in enumerate(train_classes):
                
                select_idx = train_indices[i][np.random.choice(len(train_indices[i]), num, replace=False)]
                
                split = int(num*0.7)
                
                
                train_idxs.extend(select_idx[:split])
                
                val_idxs.extend(select_idx[split:])
                
            X_val, y_val = X_train0[val_idxs] , y_train0[val_idxs]
            X_train, y_train = X_train0[train_idxs], y_train0[train_idxs], 
            
            df_train=torch.utils.data.TensorDataset(torch.tensor(X_train),torch.tensor(y_train))
            df_val=torch.utils.data.TensorDataset(torch.tensor(X_val),torch.tensor(y_val))
            df_test=torch.utils.data.TensorDataset(torch.tensor(X_test0),torch.tensor(y_test0))

            df_train = DataLoader(dataset=df_train, batch_size=batch_size, shuffle=True, drop_last=True)
            
            df_val = DataLoader(dataset=df_val, batch_size=batch_size, shuffle=True, drop_last=True)

            df_test = DataLoader(dataset=df_test, batch_size=batch_size, shuffle=True, drop_last=True)            

            ACC_1=[]
            ACC_2=[]
            LOSS1=[]
            LOSS2=[]
            save1=0
            save2=0
            for epoch in range(1, Epoch):
                for batch_idx, (data, target) in enumerate(df_train):
                    data, target = data.to(device), target.to(device)
                    a.train()
                    pre1,pre2=a(data,epoch)
        
                    loss = (criterion(pre1,target)+criterion(pre2,target)) 

                    a.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                    if batch_idx % 2000 == 0:
                        a.eval()

                        with torch.no_grad():

                            corrects1 = torch.zeros(1).to(device)
                            corrects2 = torch.zeros(1).to(device)
                            for idx, (data, target ) in enumerate(df_val):
                                data, target  = data.to(device), target.to(device)
                                pre1,pre2=a(data,epoch)
                                _, preds1 = torch.max(pre1, 1)
                                _, preds2 = torch.max(pre2, 1)
                                corrects1 += (preds1 == target).sum()
                                corrects2 += (preds2 == target).sum()
                            acc1 = corrects1.item() / len(df_val.dataset)
                            acc2 = corrects2.item() / len(df_val.dataset)

                            print('***** Eval Result epoch: {:.0f},acc1: {:.4f}, acc2: {:.4f}'.format(epoch,acc1,acc2))  
                            if acc1 > save1:
                                save1=acc1
                                save2=acc2
                                torch.save(a, './saved_plot/model save/HDMTL23.pt')
                                print('model save in 1 **************') 
                                
                            if acc2 > save2:
                                save1=acc1
                                save2=acc2
                                torch.save(a, './saved_plot/model save/HDMTL23.pt')
                                print('model save in 2 **************')  
                                           
            path_model='./saved_plot/model save/HDMTL23.pt'
            
            model =torch.load(path_model)

            model.eval()
            corrects1 = torch.zeros(1).to(device)
            corrects2 = torch.zeros(1).to(device)
            for idx, (data, target ) in enumerate(df_test):
                data, target  = data.to(device), target.to(device)
                pre1,pre2=model(data,epoch)
                _, preds1 = torch.max(pre1, 1)
                _, preds2 = torch.max(pre2, 1)
                corrects1 += (preds1 == target).sum()
                corrects2 += (preds2 == target).sum()
                
            acc1 = corrects1.item() / len(df_test.dataset)
            acc2 = corrects2.item() / len(df_test.dataset) 
            scores1.append(acc1)
            scores2.append(acc2)
                                           
            print('***** Test Result acc1: {:.4f}, acc2: {:.4f}'.format(acc1,acc2))
            
            a =pd.DataFrame(np.array(scores1))
            a.to_csv("Result_2/%s/size_%s/time_%s/scores1.csv" % (exp_name,exp,time_idx),index=True) 
            a =pd.DataFrame(np.array(scores2))
            a.to_csv("Result_2/%s/size_%s/time_%s/scores2.csv" % (exp_name,exp,time_idx),index=True)    
          
        a =pd.DataFrame(np.array(scores1))
        a.to_csv("Result_2/%s/size_%s/scores1.csv" % (exp_name,exp),index=True) 
        a =pd.DataFrame(np.array(scores2))
        a.to_csv("Result_2/%s/size_%s/scores2.csv" % (exp_name,exp),index=True) 
        
def Cross_analysis(exps_name,exps):
    
    scores_all1 = pd.DataFrame()
    scores_all2 = pd.DataFrame()
    for exp in exps:
        
        file_path = "Result_2/%s/size_%s" % (exp_name,exp)
        
        Result_2_data1 = pd.read_csv("%s/scores1.csv" % (file_path), 
                               sep=',', index_col=0)
        Result_2_data1['exp'] = exp 
        scores_all1 = pd.concat([scores_all1,Result_2_data1],axis=0) 
        
        Result_2_data2 = pd.read_csv("%s/scores2.csv" % (file_path), 
                               sep=',', index_col=0)
        Result_2_data2['exp'] = exp 
        scores_all2 = pd.concat([scores_all2,Result_2_data2],axis=0)   
        
    scores_all1.to_csv("Result_2/%s/scores_all1.csv" % (exp_name), float_format='%.6f', index=True)
    
    scores_all2.to_csv("Result_2/%s/scores_all2.csv" % (exp_name), float_format='%.6f', index=True)


    scores_all1['model'] = 'S'
    scores_all2['model'] = 'T'

    scores_all = pd.concat([scores_all1,scores_all2],axis=0)
    scores_all.to_csv("Result_2/%s/scores_all.csv" % (exp_name), float_format='%.6f', index=True)   
    
    return scores_all


In [10]:
import torch.optim as optim
import errno
batch_size=2
ASU_enabled=True
# cross_start=0
Epoch=100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
time=Time(600,2800)

settings = {
  "save_path":""
}

exp_name = "HDMTL Supplementary experiments/HDMTL 23"
exps = [90]

times = 10
pre1_list = []
L_list=[]
Cross_train_and_test(exp_name,exps)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  from ipykernel import kernelapp as app


(22007, 240)
*************HDMTL Supplementary experiments/HDMTL 23/size_90/time_0/
***** Eval Result epoch: 1,acc1: 0.1111, acc2: 0.0741


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type HDMTLNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ 

model save in 1 **************
***** Eval Result epoch: 2,acc1: 0.1111, acc2: 0.1111
model save in 2 **************
***** Eval Result epoch: 3,acc1: 0.1111, acc2: 0.1481
model save in 2 **************
***** Eval Result epoch: 4,acc1: 0.1111, acc2: 0.1852
model save in 2 **************
***** Eval Result epoch: 5,acc1: 0.2593, acc2: 0.2593
model save in 1 **************
***** Eval Result epoch: 6,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 7,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 8,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 9,acc1: 0.1852, acc2: 0.1481
***** Eval Result epoch: 10,acc1: 0.2222, acc2: 0.2222
***** Eval Result epoch: 11,acc1: 0.1852, acc2: 0.2222
***** Eval Result epoch: 12,acc1: 0.2222, acc2: 0.2593
***** Eval Result epoch: 13,acc1: 0.2222, acc2: 0.2593
***** Eval Result epoch: 14,acc1: 0.2222, acc2: 0.2222
***** Eval Result epoch: 15,acc1: 0.2593, acc2: 0.1852
***** Eval Result epoch: 16,acc1: 0.3333, acc2: 0.3333
model save in 1 ************

***** Eval Result epoch: 32,acc1: 0.3704, acc2: 0.4815
***** Eval Result epoch: 33,acc1: 0.5926, acc2: 0.5556
model save in 1 **************
***** Eval Result epoch: 34,acc1: 0.5185, acc2: 0.5556
***** Eval Result epoch: 35,acc1: 0.4815, acc2: 0.5185
***** Eval Result epoch: 36,acc1: 0.5556, acc2: 0.4815
***** Eval Result epoch: 37,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 38,acc1: 0.3333, acc2: 0.3333
***** Eval Result epoch: 39,acc1: 0.4815, acc2: 0.5185
***** Eval Result epoch: 40,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 41,acc1: 0.5185, acc2: 0.4815
***** Eval Result epoch: 42,acc1: 0.5926, acc2: 0.5185
***** Eval Result epoch: 43,acc1: 0.6296, acc2: 0.6296
model save in 1 **************
***** Eval Result epoch: 44,acc1: 0.5185, acc2: 0.5556
***** Eval Result epoch: 45,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 46,acc1: 0.5556, acc2: 0.5185
***** Eval Result epoch: 47,acc1: 0.5185, acc2: 0.4815
***** Eval Result epoch: 48,acc1: 0.5185, acc2: 0.5185
***

...

***** Eval Result epoch: 2,acc1: 0.1111, acc2: 0.1111
***** Eval Result epoch: 3,acc1: 0.1111, acc2: 0.2222
model save in 2 **************
***** Eval Result epoch: 4,acc1: 0.1111, acc2: 0.1852
***** Eval Result epoch: 5,acc1: 0.1481, acc2: 0.1481
model save in 1 **************
***** Eval Result epoch: 6,acc1: 0.2963, acc2: 0.2222
model save in 1 **************
***** Eval Result epoch: 7,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 8,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 9,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 10,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 11,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 12,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 13,acc1: 0.2593, acc2: 0.2593
model save in 2 **************
***** Eval Result epoch: 14,acc1: 0.2222, acc2: 0.1852
***** Eval Result epoch: 15,acc1: 0.2963, acc2: 0.2593
model save in 1 **************
***** Eval Result epoch: 16,acc1: 0.2963, acc2: 0.2593
***** Eval Result epoch: 17,

model save in 1 **************
***** Eval Result epoch: 31,acc1: 0.5556, acc2: 0.5556
model save in 1 **************
***** Eval Result epoch: 32,acc1: 0.5556, acc2: 0.5185
***** Eval Result epoch: 33,acc1: 0.5926, acc2: 0.6296
model save in 1 **************
***** Eval Result epoch: 34,acc1: 0.6296, acc2: 0.6296
model save in 1 **************
***** Eval Result epoch: 35,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 36,acc1: 0.5926, acc2: 0.5926
***** Eval Result epoch: 37,acc1: 0.5185, acc2: 0.5556
***** Eval Result epoch: 38,acc1: 0.6296, acc2: 0.6296
***** Eval Result epoch: 39,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 40,acc1: 0.6296, acc2: 0.6296
***** Eval Result epoch: 41,acc1: 0.5556, acc2: 0.5926
***** Eval Result epoch: 42,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 43,acc1: 0.6667, acc2: 0.6667
model save in 1 **************
***** Eval Result epoch: 44,acc1: 0.5185, acc2: 0.5556
***** Eval Result epoch: 45,acc1: 0.6296, acc2: 0.6296
***** Eval Result ep

***** Eval Result epoch: 67,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 68,acc1: 0.4444, acc2: 0.4074
***** Eval Result epoch: 69,acc1: 0.4444, acc2: 0.4444
***** Eval Result epoch: 70,acc1: 0.4074, acc2: 0.4074
***** Eval Result epoch: 71,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 72,acc1: 0.4444, acc2: 0.4444
***** Eval Result epoch: 73,acc1: 0.5926, acc2: 0.5926
model save in 1 **************
***** Eval Result epoch: 74,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 75,acc1: 0.4815, acc2: 0.4444
***** Eval Result epoch: 76,acc1: 0.4074, acc2: 0.4074
***** Eval Result epoch: 77,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 78,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 79,acc1: 0.4815, acc2: 0.4444
***** Eval Result epoch: 80,acc1: 0.4444, acc2: 0.4444
***** Eval Result epoch: 81,acc1: 0.4444, acc2: 0.4815
***** Eval Result epoch: 82,acc1: 0.4815, acc2: 0.5185
***** Eval Result epoch: 83,acc1: 0.4074, acc2: 0.4074
***** Eval Result epoch: 84,acc1: 

***** Eval Result epoch: 5,acc1: 0.2222, acc2: 0.2222
model save in 1 **************
***** Eval Result epoch: 6,acc1: 0.2222, acc2: 0.2222
***** Eval Result epoch: 7,acc1: 0.1852, acc2: 0.1852
***** Eval Result epoch: 8,acc1: 0.2593, acc2: 0.2222
model save in 1 **************
***** Eval Result epoch: 9,acc1: 0.2222, acc2: 0.2222
***** Eval Result epoch: 10,acc1: 0.2222, acc2: 0.1852
***** Eval Result epoch: 11,acc1: 0.2593, acc2: 0.2593
model save in 2 **************
***** Eval Result epoch: 12,acc1: 0.2963, acc2: 0.2593
model save in 1 **************
***** Eval Result epoch: 13,acc1: 0.3333, acc2: 0.2593
model save in 1 **************
***** Eval Result epoch: 14,acc1: 0.3333, acc2: 0.2593
***** Eval Result epoch: 15,acc1: 0.2593, acc2: 0.2222
***** Eval Result epoch: 16,acc1: 0.2963, acc2: 0.3333
model save in 2 **************
***** Eval Result epoch: 17,acc1: 0.2963, acc2: 0.2593
***** Eval Result epoch: 18,acc1: 0.3333, acc2: 0.2963
model save in 1 **************
***** Eval Result 

model save in 1 **************
***** Eval Result epoch: 35,acc1: 0.5556, acc2: 0.5926
model save in 2 **************
***** Eval Result epoch: 36,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 37,acc1: 0.3333, acc2: 0.4074
***** Eval Result epoch: 38,acc1: 0.5556, acc2: 0.5185
***** Eval Result epoch: 39,acc1: 0.4444, acc2: 0.4815
***** Eval Result epoch: 40,acc1: 0.5556, acc2: 0.5185
***** Eval Result epoch: 41,acc1: 0.5185, acc2: 0.4815
***** Eval Result epoch: 42,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 43,acc1: 0.5556, acc2: 0.5556
***** Eval Result epoch: 44,acc1: 0.5185, acc2: 0.5556
***** Eval Result epoch: 45,acc1: 0.5926, acc2: 0.5926
model save in 1 **************
***** Eval Result epoch: 46,acc1: 0.4815, acc2: 0.4815
***** Eval Result epoch: 47,acc1: 0.5185, acc2: 0.5185
***** Eval Result epoch: 48,acc1: 0.5926, acc2: 0.5556
***** Eval Result epoch: 49,acc1: 0.5556, acc2: 0.5556
***** Eval Result epoch: 50,acc1: 0.4444, acc2: 0.4444
***** Eval Result epoch: 51

In [11]:
# exp_name = "HDMTL Supplementary experiments/HDMTL 21"
# exps = [90]
scores_all = Cross_analysis(exp_name,exps)
scores_all_mean = scores_all.groupby(['model','exp']).mean()
scores_all_std = scores_all.groupby(['model','exp']).std()
scores_all_mean.to_csv("Result_2/%s_all_mean.csv" % (exp_name), float_format='%.4f', index=True)
scores_all_std.to_csv("Result_2/%s_all_std.csv" % (exp_name), float_format='%.4f', index=True)
# scores_all_std = scores_all_std.astype(tf.float32)
scores_all_mean,scores_all_std

(                  0
 model exp          
 S     90   0.585825
 T     90   0.586324,
                   0
 model exp          
 S     90   0.048177
 T     90   0.042717)